In [1]:
from itertools import product
import os

In [2]:
class CNF:
    def __init__(self):
        self.clauses = []
        self.number_to_var = {}
        self.var_to_number = {}
        
    def add_clause(self, clause):
        self.clauses.append(clause)
        
        for literal in clause:
            var = literal.strip('-')
            if var not in self.var_to_number:
                var_number = len(self.var_to_number) + 1
                self.var_to_number[var] = var_number
                self.number_to_var[var_number] = var
                
    def dimacs(self):
        result = f"p cnf {len(self.var_to_number)} {len(self.clauses)}\n"
        for clause in self.clauses:
            for literal in clause:
                var = literal.strip('-')
                if literal[0] == '-':
                    result += '-'
                result += f"{self.var_to_number[var]} "
            result += "0\n"
        
        return result
    
    def get_var_name(self, var_number):
        return self.number_to_var[var_number]
    
    def get_var_number(self, var_name):
        return self.var_to_number[var_name]
        

In [3]:
def minisat_solve(problem_name, cnf):
    with open(f"{problem_name}.cnf", 'w') as input_file:
        input_file.write(cnf.dimacs())
        
    os.system(f"minisat {problem_name}.cnf {problem_name}_result.cnf")
    
    with open(f"{problem_name}_result.cnf", 'r') as output_file:
        lines = output_file.readlines()
        
    if lines[0].startswith("SAT"):
        var_values = {}
        literals = lines[1].split(' ')[:-1]
        for literal in literals:
            var_number = int(literal.strip('-'))
            var_name = cnf.get_var_name(var_number)
            var_values[var_name] = 0 if literal.startswith('-') else 1
        
        true_vars = list(filter(lambda x: x[1] == 1, var_values.items()))
        
        return true_vars

In [4]:
def same_square(x1, y1, x2, y2):
    if (x1 // 3 == x2 // 3) and (y1 // 3 == y2 // 3):
        return True
    else:
        return False

In [9]:
def solve_sudoku(board):
    
    n = len(board)
    cnf = CNF()
    
    for x, y in product(range(n), repeat = 2):
        if board[x][y] != 0:
            cnf.add_clause([f"S_{x}_{y}_{board[x][y]}"])
            
    for x,y in product(range(n), repeat = 2):
        clause = [f"S_{x}_{y}_{number}" for number in range(1, n+1)]
        cnf.add_clause(clause)
        
    for x1,y1,x2,y2,number in product(range(n), repeat = 5):
        number += 1
        
        if (x1 == x2 and y1 < y2) \
        or (y1 == y2 and x1 < x2) \
        or ( (x1,y1) != (x2, y2) and same_square(x1, y1, x2, y2)):
            cnf.add_clause([f"-S_{x1}_{y1}_{number}", f"-S_{x2}_{y2}_{number}"])
        
    true_vars = minisat_solve("sudoku", cnf)
    for var_name,_ in true_vars:
        x = int(var_name[2])
        y = int(var_name[4])
        number = int(var_name[6])
        
        board[x][y] = number
    
    for row in board:
        print(row)

In [12]:
board1 = [
    [0,0,4,0,5,0,0,0,0],
    [9,0,0,7,3,4,6,0,0],
    [0,0,3,0,2,1,0,4,9],
    [0,3,5,0,9,0,4,8,0],
    [0,9,0,0,0,0,0,3,0],
    [0,7,6,0,1,0,9,2,0],
    [3,1,0,9,7,0,2,0,0],
    [0,0,9,1,8,2,0,0,3],
    [0,0,0,0,6,0,1,0,0]
]
board2 = [
    [5,0,9,4,0,0,0,0,0],
    [0,0,3,0,0,0,6,9,0],
    [0,1,0,0,0,0,0,0,5],
    [0,5,0,1,8,0,0,0,0],
    [3,0,0,0,5,0,0,0,7],
    [0,0,0,0,9,6,0,5,0],
    [9,0,0,0,0,0,0,7,0],
    [0,3,8,0,0,0,5,0,0],
    [0,0,0,0,0,7,1,0,3]
]
solve_sudoku(board2)

============================[ Problem Statistics ]=============================
|                                                                             |
|  Number of variables:           729                                         |
|  Number of clauses:            4418                                         |
|  Parse time:                   0.00 s                                       |
|  Eliminated vars:               132                                           |
|  Vars set       :               461                                           |
|  Eliminated clauses:           0.00 Mb                                      |
|  Simplification time:          0.00 s                                       |
|                                                                             |
============================[ Search Statistics ]==============================
| Conflicts |          ORIGINAL         |          LEARNT          | Progress |
|           |    Vars  Clauses Liter